# How do $S$ and $\log R^\prime_{HK}$ vary within bins of spectral type?

In [ ]:
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import numpy as np
import matplotlib.pyplot as plt
from matplotlib.gridspec import GridSpec

from glob import glob
from tqdm import tqdm
import pandas as pd
import os

import astropy.units as u
from astropy.table import Table
from astropy.time import Time

from astroquery.nasa_exoplanet_archive import NasaExoplanetArchive

In [ ]:
owls_sindices = pd.read_pickle('data/db.pkl')

In [ ]:
url = (
    'https://docs.google.com/spreadsheets/d/'
    '11Z7B76FXBkEwcGmhp72sC6AQdP8ER8K_eU5RAW8ed2M'
    '/gviz/tq?tqx=out:csv&sheet=catalog'
)
owls_sheet = pd.read_csv(url)

In [ ]:
notes_column = owls_sheet['Unnamed: 11']
owls_sheet_baliunas = owls_sheet#[notes_column == 'Baliunas']

In [ ]:
bins = [0.5 , 0.75, 1.  , 1.7 , 5.  ]
nbins = len(bins) - 1
cut = pd.cut(owls_sheet_baliunas['BP'].fillna(0) - owls_sheet_baliunas['RP'].fillna(0), bins)
owls_sheet_baliunas['color_bins'] = cut

In [ ]:
gs = GridSpec(3*nbins, 1)
fig = plt.figure(figsize=(8, 10))

s0 = fig.add_subplot(gs[0])
r0 = fig.add_subplot(gs[1])

ax = [s0, r0] + [fig.add_subplot(gs[i]) for i in range(2, 3*nbins)]

for j in range(3):
    for i, spt in enumerate("F G K M".split()):
        group = owls_sheet[[s.startswith(spt) if (not str(s) == 'nan') else False for s in owls_sheet['sp_type_mamajek']]]

        colors = group['BP'] - group['RP']
        sptype = group['SP_TYPE']
        log_rprime = group['log R\'_HK']
        counter = 0
        target_labels = []
        target_sinds = []
        target_sptypes = []
        target_flags = []
        target_colors = []
        target_rprime = []
        baliunas_flag = group['Unnamed: 11'] == 'Baliunas'

        for target, c, s, f, r in zip(group['pl_hostname'], colors, sptype, baliunas_flag, log_rprime):
            if target in owls_sindices.index.get_level_values(0):
                counter += 1
                sinds = owls_sindices.loc[target]['S']
                target_sinds.append(sinds)
                target_labels.append(target)
                target_sptypes.append(s)
                target_flags.append(f)
                target_colors.append(c)
                target_rprime.append(r)

        sort = np.argsort(target_colors)
        y = np.array([ts.values for ts in target_sinds], dtype=object)[sort]
        sorted_labels = np.array(target_labels, dtype=object)[sort]
        sorted_spt = np.array(target_sptypes, dtype=object)[sort]
        sorted_flags = np.array(target_flags, dtype=object)[sort]
        sorted_rprime = np.array(target_rprime, dtype=object)[sort]
        
        ax_ind = 3*i + j
        
        xticks = []
        xticklabels = []
        sptticklabels = []
        for c, yi, name, s, f, r in zip(np.arange(counter), y, sorted_labels, sorted_spt, sorted_flags, sorted_rprime):
            if j == 0:
                # Plot S indices
                ax[ax_ind].semilogy(c * np.ones_like(yi), yi, 'o' if not f else 's', ms=5, color='k' if not f else 'r', rasterized=True)
            elif j == 1: 
                # Plot Rprime
                ax[ax_ind].plot(c * np.ones_like(r), r, 'o' if not f else 's', ms=5, color='k' if not f else 'r', rasterized=True)

            xticks.append(c)
            xticklabels.append(name)
            sptticklabels.append(s)

        # Egeland 2017 solar S-index range
        if j == 0:
            ax[ax_ind].set_title(spt)
            ax[ax_ind].set_xticks(xticks)
            ax[ax_ind].set_ylim([0.1, 20])
            ax[ax_ind].axhspan(0.16, 0.18, alpha=0.2)
            ax[ax_ind].set_ylabel(r'$S$')
            ax[ax_ind].set_xticklabels([])

        elif j == 1:
            ax[ax_ind].axhspan(-4.98, -4.9, alpha=0.2)
            ax[ax_ind].set_ylim([-5.1, -3])
            ax[ax_ind].set_ylabel(r'$\log R^\prime_{{HK}}$')
            ax[ax_ind].set_xticks(xticks)
            ax[ax_ind].set_xticklabels(xticklabels, rotation=90, ha='center', fontsize=4)
            
        elif ax_ind < len(ax): 
            ax[ax_ind].axis('off')

fig.savefig('plots/activity_spectral_sequence.pdf', dpi=200, bbox_inches='tight')